In [3]:
# -*- coding: utf-8 -*-
"""Training_Sony.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1GEBhrMgdTE81EYLi2hUD6Xrie9iRhLZg
"""
from __future__ import division
import os, time

#pip install scipy==1.2.0

import scipy as scipy

# uniform content loss + adaptive threshold + per_class_input + recursive G
# improvement upon cqf37

#from google.colab import drive
#drive.mount('/content/drive',force_remount = True)

import tensorflow.compat.v1 as tf
#import tensorflow.contrib.slim as slim
import numpy as np

#!pip install --upgrade tf_slim

import tf_slim as slim

import sys

#pip install rawpy

import rawpy as rawpy
import glob
from PIL import Image


In [5]:
input_dir = '/home/21i190003/DL_Project/Sony_Dataset/Sony/short/'
gt_dir = '/home/21i190003/DL_Project/Sony_Dataset/Sony/long/'
checkpoint_dir = '/home/21i190003/DL_Project/Sony_ckpt1/'
result_dir = '/home/21i190003/DL_Project/Sony_result1/'
print('------------------------------------------------------------------\n------------------------------------------------------------------\n------------------------------------------------------------------')        

# get train IDs
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

print(f'No. of Training Images: {len(train_ids)}') #Number of reference images for training for Sony Sensor

ps = 512  # patch size for training
save_freq = 5

DEBUG = 0
if DEBUG == 1:
    save_freq = 1
    train_ids = train_ids[0:5]


------------------------------------------------------------------
------------------------------------------------------------------
------------------------------------------------------------------
No. of Training Images: 0


In [1]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)

def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 2)
    return out


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out


In [6]:
print('Initiating Session....')
init_op = tf.initialize_all_variables()
sess = tf.Session()
tf.disable_eager_execution()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
tf.disable_v2_behavior()
out_image = network(in_image)

G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))

t_vars = tf.trainable_variables()
lr = tf.placeholder(tf.float32)
G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)


# Raw data takes long time to load. Keep them in memory after loaded.
gt_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')


Initiating Session....


ValueError: Variable g_conv1_1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tf_slim/ops/variables.py", line 244, in variable
    return getter(
  File "/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tf_slim/ops/arg_scope.py", line 184, in func_with_args
    return func(*args, **current_args)
  File "/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tf_slim/ops/variables.py", line 314, in model_variable
    var = variable(
  File "/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tf_slim/ops/arg_scope.py", line 184, in func_with_args
    return func(*args, **current_args)
  File "/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tf_slim/layers/layers.py", line 1771, in _model_variable_getter
    return variables.model_variable(


In [6]:
print('Training Started------------------------------------------------------------------\n------------------------------------------------------------------\n------------------------------------------------------------------')        
np.random.seed(10)
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))

import warnings
warnings.filterwarnings("ignore")

learning_rate = 1e-4
for epoch in range(lastepoch, 520):
    if os.path.isdir(result_dir + '%04d' % epoch):
        continue
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5

    for ind in np.random.permutation(len(train_ids)):
        # get the path from image id
        train_id = train_ids[ind]
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
        in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
        in_fn = os.path.basename(in_path)

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        st = time.time()
        cnt += 1

        if input_images[str(ratio)[0:3]][ind] is None:
            #try:
                raw = rawpy.imread(in_path)
                input_images[str(ratio)[0:3]][ind] = np.expand_dims(pack_raw(raw), axis=0) * ratio

                gt_raw = rawpy.imread(gt_path)
                im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)
        # crop
        H = input_images[str(ratio)[0:3]][ind].shape[1]
        W = input_images[str(ratio)[0:3]][ind].shape[2]

        xx = np.random.randint(0, W - ps)
        yy = np.random.randint(0, H - ps)
        input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps, xx:xx + ps, :]
        gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]

        if np.random.randint(2, size=1)[0] == 1:  # random flip
            input_patch = np.flip(input_patch, axis=1)
            gt_patch = np.flip(gt_patch, axis=1)
        if np.random.randint(2, size=1)[0] == 1:
                    input_patch = np.flip(input_patch, axis=2)
                    gt_patch = np.flip(gt_patch, axis=2)
        if np.random.randint(2, size=1)[0] == 1:  # random transpose
                    input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                    gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

        input_patch = np.minimum(input_patch, 1.0)

        _, G_current, output = sess.run([G_opt, G_loss, out_image],
                                                feed_dict={in_image: input_patch, gt_image: gt_patch, lr: learning_rate})
        output = np.minimum(np.maximum(output, 0), 1)
        g_loss[ind] = G_current
        #print(g_loss[np.where(g_loss)])
        print("%d , Count:  %d , Train ID: %d , Loss=%.3f , Time=%.3f" % ((epoch+1), cnt,train_id, np.mean(g_loss[np.where(g_loss)]), time.time() - st))

        if epoch % save_freq == 0:
            if not os.path.isdir(result_dir + '%04d' % epoch):
                os.makedirs(result_dir + '%04d' % epoch)

            temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :, :]), axis=1)
            Image.fromarray((temp * 255).astype(np.uint8),mode='RGB').save(
                        result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))        
        #except:
        #    print(f"Oops! {sys.exc_info()[0]} occurred for Train ID {train_id}, moving to next training ID....")

        #if cnt>=100:
        #    break
    print(f'------------------------------------------------------------------\nEpoch: {epoch+1} to Epoch: {epoch+2}------------------------------------------------------------------\n------------------------------------------------------------------')        
    if not os.path.isdir(result_dir + 'model.ckpt'):
      os.makedirs(result_dir + 'model.ckpt')
    saver.save(sess, checkpoint_dir + 'model.ckpt')


Training Started------------------------------------------------------------------
------------------------------------------------------------------
------------------------------------------------------------------
512 , Count:  1 , Train ID: 42 , Loss=0.016 , Time=3.607
512 , Count:  2 , Train ID: 24 , Loss=0.021 , Time=2.973
512 , Count:  3 , Train ID: 150 , Loss=0.025 , Time=2.960
512 , Count:  4 , Train ID: 44 , Loss=0.024 , Time=2.962
512 , Count:  5 , Train ID: 196 , Loss=0.024 , Time=2.964
512 , Count:  6 , Train ID: 97 , Loss=0.024 , Time=2.946
512 , Count:  7 , Train ID: 118 , Loss=0.026 , Time=2.902
512 , Count:  8 , Train ID: 85 , Loss=0.030 , Time=2.884
512 , Count:  9 , Train ID: 17 , Loss=0.028 , Time=2.971
512 , Count:  10 , Train ID: 181 , Loss=0.031 , Time=2.859
512 , Count:  11 , Train ID: 38 , Loss=0.031 , Time=2.919
512 , Count:  12 , Train ID: 138 , Loss=0.032 , Time=2.897
512 , Count:  13 , Train ID: 15 , Loss=0.031 , Time=2.953
512 , Count:  14 , Train ID: 123 

512 , Count:  137 , Train ID: 169 , Loss=0.028 , Time=2.880
512 , Count:  138 , Train ID: 183 , Loss=0.028 , Time=2.848
512 , Count:  139 , Train ID: 218 , Loss=0.028 , Time=2.896
512 , Count:  140 , Train ID: 47 , Loss=0.028 , Time=2.893
512 , Count:  141 , Train ID: 204 , Loss=0.028 , Time=2.985
512 , Count:  142 , Train ID: 117 , Loss=0.028 , Time=2.855
512 , Count:  143 , Train ID: 119 , Loss=0.028 , Time=2.925
512 , Count:  144 , Train ID: 166 , Loss=0.028 , Time=2.876
512 , Count:  145 , Train ID: 12 , Loss=0.028 , Time=2.897
512 , Count:  146 , Train ID: 224 , Loss=0.028 , Time=2.881
512 , Count:  147 , Train ID: 152 , Loss=0.028 , Time=2.948
512 , Count:  148 , Train ID: 37 , Loss=0.028 , Time=2.838
512 , Count:  149 , Train ID: 220 , Loss=0.028 , Time=2.928
512 , Count:  150 , Train ID: 53 , Loss=0.028 , Time=2.899
512 , Count:  151 , Train ID: 13 , Loss=0.028 , Time=2.908
512 , Count:  152 , Train ID: 60 , Loss=0.028 , Time=2.921
512 , Count:  153 , Train ID: 52 , Loss=0.028 

513 , Count:  112 , Train ID: 64 , Loss=0.029 , Time=3.070
513 , Count:  113 , Train ID: 29 , Loss=0.030 , Time=2.864
513 , Count:  114 , Train ID: 73 , Loss=0.030 , Time=1.918
513 , Count:  115 , Train ID: 95 , Loss=0.030 , Time=1.943
513 , Count:  116 , Train ID: 12 , Loss=0.030 , Time=2.947
513 , Count:  117 , Train ID: 98 , Loss=0.030 , Time=1.895
513 , Count:  118 , Train ID: 21 , Loss=0.030 , Time=1.913
513 , Count:  119 , Train ID: 57 , Loss=0.030 , Time=2.916
513 , Count:  120 , Train ID: 58 , Loss=0.030 , Time=2.969
513 , Count:  121 , Train ID: 124 , Loss=0.030 , Time=1.990
513 , Count:  122 , Train ID: 99 , Loss=0.030 , Time=2.078
513 , Count:  123 , Train ID: 100 , Loss=0.030 , Time=2.009
513 , Count:  124 , Train ID: 62 , Loss=0.030 , Time=3.075
513 , Count:  125 , Train ID: 14 , Loss=0.030 , Time=2.063
513 , Count:  126 , Train ID: 152 , Loss=0.030 , Time=2.089
513 , Count:  127 , Train ID: 119 , Loss=0.030 , Time=1.894
513 , Count:  128 , Train ID: 171 , Loss=0.030 , Tim

514 , Count:  87 , Train ID: 53 , Loss=0.030 , Time=1.965
514 , Count:  88 , Train ID: 51 , Loss=0.030 , Time=2.082
514 , Count:  89 , Train ID: 205 , Loss=0.030 , Time=2.024
514 , Count:  90 , Train ID: 196 , Loss=0.030 , Time=2.932
514 , Count:  91 , Train ID: 78 , Loss=0.030 , Time=2.051
514 , Count:  92 , Train ID: 161 , Loss=0.030 , Time=1.945
514 , Count:  93 , Train ID: 131 , Loss=0.030 , Time=2.033
514 , Count:  94 , Train ID: 31 , Loss=0.030 , Time=2.933
514 , Count:  95 , Train ID: 138 , Loss=0.030 , Time=2.003
514 , Count:  96 , Train ID: 194 , Loss=0.029 , Time=3.061
514 , Count:  97 , Train ID: 164 , Loss=0.029 , Time=1.995
514 , Count:  98 , Train ID: 12 , Loss=0.029 , Time=1.910
514 , Count:  99 , Train ID: 56 , Loss=0.029 , Time=2.000
514 , Count:  100 , Train ID: 119 , Loss=0.029 , Time=1.970
514 , Count:  101 , Train ID: 46 , Loss=0.030 , Time=2.001
514 , Count:  102 , Train ID: 104 , Loss=0.030 , Time=2.034
514 , Count:  103 , Train ID: 71 , Loss=0.030 , Time=1.944
5

515 , Count:  62 , Train ID: 183 , Loss=0.030 , Time=1.990
515 , Count:  63 , Train ID: 75 , Loss=0.030 , Time=1.969
515 , Count:  64 , Train ID: 42 , Loss=0.030 , Time=2.022
515 , Count:  65 , Train ID: 138 , Loss=0.030 , Time=2.003
515 , Count:  66 , Train ID: 104 , Loss=0.030 , Time=1.971
515 , Count:  67 , Train ID: 230 , Loss=0.030 , Time=2.064
515 , Count:  68 , Train ID: 219 , Loss=0.030 , Time=2.952
515 , Count:  69 , Train ID: 71 , Loss=0.030 , Time=1.955
515 , Count:  70 , Train ID: 23 , Loss=0.030 , Time=2.042
515 , Count:  71 , Train ID: 15 , Loss=0.030 , Time=1.989
515 , Count:  72 , Train ID: 96 , Loss=0.030 , Time=1.941
515 , Count:  73 , Train ID: 48 , Loss=0.030 , Time=2.008
515 , Count:  74 , Train ID: 60 , Loss=0.030 , Time=1.983
515 , Count:  75 , Train ID: 39 , Loss=0.030 , Time=1.984
515 , Count:  76 , Train ID: 112 , Loss=0.030 , Time=1.983
515 , Count:  77 , Train ID: 97 , Loss=0.030 , Time=2.009
515 , Count:  78 , Train ID: 197 , Loss=0.030 , Time=1.967
515 , C

517 , Count:  37 , Train ID: 154 , Loss=0.030 , Time=2.053
517 , Count:  38 , Train ID: 143 , Loss=0.030 , Time=1.946
517 , Count:  39 , Train ID: 58 , Loss=0.030 , Time=2.015
517 , Count:  40 , Train ID: 37 , Loss=0.030 , Time=2.020
517 , Count:  41 , Train ID: 71 , Loss=0.030 , Time=1.968
517 , Count:  42 , Train ID: 81 , Loss=0.030 , Time=2.021
517 , Count:  43 , Train ID: 150 , Loss=0.030 , Time=2.026
517 , Count:  44 , Train ID: 56 , Loss=0.030 , Time=2.947
517 , Count:  45 , Train ID: 127 , Loss=0.030 , Time=1.946
517 , Count:  46 , Train ID: 85 , Loss=0.030 , Time=2.103
517 , Count:  47 , Train ID: 219 , Loss=0.030 , Time=2.129
517 , Count:  48 , Train ID: 231 , Loss=0.030 , Time=2.976
517 , Count:  49 , Train ID: 18 , Loss=0.030 , Time=2.019
517 , Count:  50 , Train ID: 47 , Loss=0.030 , Time=2.039
517 , Count:  51 , Train ID: 149 , Loss=0.030 , Time=1.987
517 , Count:  52 , Train ID: 189 , Loss=0.030 , Time=1.990
517 , Count:  53 , Train ID: 15 , Loss=0.030 , Time=2.954
517 , 

518 , Count:  12 , Train ID: 194 , Loss=0.029 , Time=3.047
518 , Count:  13 , Train ID: 181 , Loss=0.029 , Time=1.947
518 , Count:  14 , Train ID: 37 , Loss=0.029 , Time=1.957
518 , Count:  15 , Train ID: 154 , Loss=0.029 , Time=2.047
518 , Count:  16 , Train ID: 81 , Loss=0.029 , Time=1.952
518 , Count:  17 , Train ID: 36 , Loss=0.029 , Time=2.000
518 , Count:  18 , Train ID: 160 , Loss=0.029 , Time=1.908
518 , Count:  19 , Train ID: 131 , Loss=0.029 , Time=1.942
518 , Count:  20 , Train ID: 59 , Loss=0.029 , Time=2.069
518 , Count:  21 , Train ID: 205 , Loss=0.029 , Time=2.011
518 , Count:  22 , Train ID: 88 , Loss=0.029 , Time=2.018
518 , Count:  23 , Train ID: 21 , Loss=0.029 , Time=2.044
518 , Count:  24 , Train ID: 204 , Loss=0.029 , Time=1.923
518 , Count:  25 , Train ID: 135 , Loss=0.029 , Time=2.011
518 , Count:  26 , Train ID: 10 , Loss=0.029 , Time=1.929
518 , Count:  27 , Train ID: 19 , Loss=0.030 , Time=1.929
518 , Count:  28 , Train ID: 142 , Loss=0.030 , Time=1.929
518 ,

518 , Count:  152 , Train ID: 196 , Loss=0.029 , Time=1.921
518 , Count:  153 , Train ID: 161 , Loss=0.029 , Time=2.011
518 , Count:  154 , Train ID: 15 , Loss=0.029 , Time=2.050
518 , Count:  155 , Train ID: 98 , Loss=0.029 , Time=2.033
518 , Count:  156 , Train ID: 128 , Loss=0.030 , Time=1.959
518 , Count:  157 , Train ID: 108 , Loss=0.030 , Time=2.028
518 , Count:  158 , Train ID: 9 , Loss=0.030 , Time=2.011
518 , Count:  159 , Train ID: 169 , Loss=0.030 , Time=1.985
518 , Count:  160 , Train ID: 17 , Loss=0.030 , Time=1.988
518 , Count:  161 , Train ID: 67 , Loss=0.030 , Time=2.063
------------------------------------------------------------------
Epoch: 518 to Epoch: 519------------------------------------------------------------------
------------------------------------------------------------------
519 , Count:  1 , Train ID: 88 , Loss=0.030 , Time=2.019
519 , Count:  2 , Train ID: 91 , Loss=0.030 , Time=1.973
519 , Count:  3 , Train ID: 194 , Loss=0.029 , Time=2.069
519 , Cou

519 , Count:  127 , Train ID: 221 , Loss=0.030 , Time=1.982
519 , Count:  128 , Train ID: 49 , Loss=0.030 , Time=1.985
519 , Count:  129 , Train ID: 158 , Loss=0.030 , Time=1.899
519 , Count:  130 , Train ID: 27 , Loss=0.030 , Time=2.029
519 , Count:  131 , Train ID: 60 , Loss=0.030 , Time=1.985
519 , Count:  132 , Train ID: 31 , Loss=0.030 , Time=1.975
519 , Count:  133 , Train ID: 70 , Loss=0.030 , Time=1.991
519 , Count:  134 , Train ID: 195 , Loss=0.030 , Time=1.954
519 , Count:  135 , Train ID: 29 , Loss=0.030 , Time=1.973
519 , Count:  136 , Train ID: 181 , Loss=0.030 , Time=1.964
519 , Count:  137 , Train ID: 231 , Loss=0.030 , Time=2.070
519 , Count:  138 , Train ID: 10 , Loss=0.030 , Time=2.109
519 , Count:  139 , Train ID: 47 , Loss=0.030 , Time=1.956
519 , Count:  140 , Train ID: 136 , Loss=0.030 , Time=2.085
519 , Count:  141 , Train ID: 171 , Loss=0.030 , Time=1.973
519 , Count:  142 , Train ID: 96 , Loss=0.030 , Time=2.091
519 , Count:  143 , Train ID: 64 , Loss=0.030 , T

520 , Count:  102 , Train ID: 137 , Loss=0.030 , Time=1.997
520 , Count:  103 , Train ID: 78 , Loss=0.030 , Time=2.074
520 , Count:  104 , Train ID: 225 , Loss=0.030 , Time=2.121
520 , Count:  105 , Train ID: 15 , Loss=0.030 , Time=1.997
520 , Count:  106 , Train ID: 41 , Loss=0.030 , Time=2.147
520 , Count:  107 , Train ID: 113 , Loss=0.030 , Time=2.084
520 , Count:  108 , Train ID: 2 , Loss=0.030 , Time=2.169
520 , Count:  109 , Train ID: 37 , Loss=0.030 , Time=2.116
520 , Count:  110 , Train ID: 157 , Loss=0.030 , Time=1.964
520 , Count:  111 , Train ID: 154 , Loss=0.031 , Time=2.120
520 , Count:  112 , Train ID: 132 , Loss=0.031 , Time=1.951
520 , Count:  113 , Train ID: 31 , Loss=0.031 , Time=2.011
520 , Count:  114 , Train ID: 92 , Loss=0.031 , Time=1.979
520 , Count:  115 , Train ID: 124 , Loss=0.031 , Time=1.971
520 , Count:  116 , Train ID: 52 , Loss=0.030 , Time=2.018
520 , Count:  117 , Train ID: 64 , Loss=0.030 , Time=1.960
520 , Count:  118 , Train ID: 91 , Loss=0.030 , Ti